<a href="https://colab.research.google.com/github/ShirinPullot/podcast_summarization/blob/main/Podify_Summarization_of_podcasts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun Mar 19 08:08:48 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P0    29W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from __future__ import unicode_literals


In [3]:
#install python 3.9
!sudo apt-get update -y
!sudo apt-get install python3.9

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:7 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:10 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Hit:11 http://security.ubuntu.com/ubuntu focal-security InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3.9 is already the newest version (3.9.16-1+focal1).
0 upgraded, 0 newly installed, 0 to remove a

In [4]:
! pip install openai-whisper==20230117

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import re
import whisper
from whisper.utils import write_vtt
import requests
import os
import nltk
import heapq
from bs4 import BeautifulSoup

nltk.download('punkt')
nltk.download('stopwords')




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True


# 1. First step is to get transcripts from podcast link


In [6]:


model = whisper.load_model("medium")

def get_transcripts(link):
  content = requests.get(link)
  soup = BeautifulSoup(content.text)
  podcast_url = re.findall("(?P<url>\;https?://[^\s]+)", content.text)[0].split(';')[1]
  print(podcast_url)
  

  download = requests.get(podcast_url)

  with open('podcast.mp3', 'wb') as f:
    f.write(download.content)

  result = model.transcribe('podcast.mp3')

  with open('sub.vtt', "w", encoding='utf-8') as txt:
    write_vtt(result["segments"], file=txt)

  return (result['text'], 'sub.vtt')

  




In [33]:
transcripts= get_transcripts('https://podcasts.google.com/feed/aHR0cHM6Ly9mZWVkczIuZmVlZGJ1cm5lci5jb20vdGVkdGFsa3NfYXVkaW8/episode/ZW4uYXVkaW8udGFsay50ZWQuY29tOjk3NDA4?sa=X&ved=0CAUQkfYCahgKEwiQuOnlr7L6AhUAAAAAHQAAAAAQiwc')
transcript= ''.join(transcripts)
print(transcript)

https://backend.endpoints.firstory-709db.cloud.goog/play.mp3?url=https%3A%2F%2Fdts.podtrac.com%2Fredirect.mp3%2Fdownload.ted.com%2Ftalks%2FEnzoRomero_2022U.mp3%3Fapikey%3D172BB350-0207%26prx_url%3Dhttps%3A%2F%2Fchtbl.com%2Ftrack%2F48D18%2Fhttps%3A%2F%2Fdovetail.prxu.org%2F70%2F1ebf1923-1f21-455d-9520-2319f2b9f17c%2FEnzoRomero_2022U_VO_Intro.mp3
 It's TED Talks Daily. I'm Elise Hugh. Prosthetics have advanced so much in recent years, but for amputees around the world, they're still often way too expensive. In his talk from TED 2022, bionic innovator Enzo Romero shares how he's found unconventional solutions to get more advanced prosthetics to the people who need them. He makes them more affordable and functional. Have you ever wondered what makes great minds tick? I'm Adam Grant, and on my new podcast Rethinking, I'm trying to find the answers. Every week, I interview some of my favorite thinkers to learn how we can bring out the best in ourselves and others. I talk to death-defying roc


# 2.Summarizing the transcripts obtained from the audio files



1.  Summarization technique using word frequency and NLTK library:



In [8]:
# def get_nltk_summary(link):
  
#   transcript= get_transcripts(link)
#   transcript= ''.join(transcript)
#   sentence_list = nltk.sent_tokenize(transcript)
#   stopwords = nltk.corpus.stopwords.words('english')
#   word_frequencies = {}
#   for word in nltk.word_tokenize(transcript):
#       if word not in stopwords:
#           if word not in word_frequencies.keys():
#               word_frequencies[word] = 1
#           else:
#               word_frequencies[word] += 1
#   maximum_frequncy = max(word_frequencies.values())
#   for word in word_frequencies.keys():
#       word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)
#   sentence_scores = {}
#   for sent in sentence_list:
#       for word in nltk.word_tokenize(sent.lower()):
#           if word in word_frequencies.keys():
#               if len(sent.split(' ')) < 30:
#                   if sent not in sentence_scores.keys():
#                       sentence_scores[sent] = word_frequencies[word]
#                   else:
#                       sentence_scores[sent] += word_frequencies[word]
#   from pprint import pprint
#   summary_sentences = heapq.nlargest(15, sentence_scores, key=sentence_scores.get)
#   summary = ' '.join(summary_sentences)
#   # pprint( summary)

#   print('lines in summary: ', len(summary.split('.')))

## 2. Summarization using genism(Text rank algorithm)

**bold text**


In [9]:
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords

In [10]:
def get_textrank_summary(link):
  transcript= get_transcripts(link)
  transcript= ''.join(transcript)
  return str(summarize(transcript, ratio=0.5))


In [34]:
textrank_summary= get_textrank_summary('https://podcasts.google.com/feed/aHR0cHM6Ly9mZWVkczIuZmVlZGJ1cm5lci5jb20vdGVkdGFsa3NfYXVkaW8/episode/ZW4uYXVkaW8udGFsay50ZWQuY29tOjk3NDA4?sa=X&ved=0CAUQkfYCahgKEwiQuOnlr7L6AhUAAAAAHQAAAAAQiwc')
print(textrank_summary)

https://backend.endpoints.firstory-709db.cloud.goog/play.mp3?url=https%3A%2F%2Fdts.podtrac.com%2Fredirect.mp3%2Fdownload.ted.com%2Ftalks%2FEnzoRomero_2022U.mp3%3Fapikey%3D172BB350-0207%26prx_url%3Dhttps%3A%2F%2Fchtbl.com%2Ftrack%2F48D18%2Fhttps%3A%2F%2Fdovetail.prxu.org%2F70%2F1ebf1923-1f21-455d-9520-2319f2b9f17c%2FEnzoRomero_2022U_VO_Intro.mp3
Prosthetics have advanced so much in recent years, but for amputees around the world, they're still often way too expensive.
In his talk from TED 2022, bionic innovator Enzo Romero shares how he's found unconventional solutions to get more advanced prosthetics to the people who need them.
I'm Adam Grant, and on my new podcast Rethinking, I'm trying to find the answers.
Find Rethinking on Apple Podcasts, Spotify, Amazon Music, or wherever you listen.
One in seven people around the world has some kind of disability.
But then being twice as stunned when the medical robots fitted in with a prosthesis that could move and feel.
From then on, no greate

In [14]:
len(transcript.split('.'))

64

## 3. Summarization using LSA :
Latent Semantic Analysis is a unsupervised learning algorithm that can be used for extractive text summarization.

It extracts semantically significant sentences by applying singular value decomposition(SVD) to the matrix of term-document frequency.

In [16]:
! pip install sumy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
from sumy.summarizers.lsa import LsaSummarizer
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser


In [18]:
def get_lsa_summary(link):
  transcript= get_transcripts(link)
  transcript= ''.join(transcript)
  parser=PlaintextParser.from_string(transcript,Tokenizer('english'))
    # creating the summarizer
  lsa_summarizer=LsaSummarizer()
  return str(lsa_summarizer(parser.document, 32))
  # Printing the summary




In [36]:
lsa_summary= get_lsa_summary('https://podcasts.google.com/feed/aHR0cHM6Ly9mZWVkczIuZmVlZGJ1cm5lci5jb20vdGVkdGFsa3NfYXVkaW8/episode/ZW4uYXVkaW8udGFsay50ZWQuY29tOjk3NDA4?sa=X&ved=0CAUQkfYCahgKEwiQuOnlr7L6AhUAAAAAHQAAAAAQiwc')
lsa_summary

https://backend.endpoints.firstory-709db.cloud.goog/play.mp3?url=https%3A%2F%2Fdts.podtrac.com%2Fredirect.mp3%2Fdownload.ted.com%2Ftalks%2FEnzoRomero_2022U.mp3%3Fapikey%3D172BB350-0207%26prx_url%3Dhttps%3A%2F%2Fchtbl.com%2Ftrack%2F48D18%2Fhttps%3A%2F%2Fdovetail.prxu.org%2F70%2F1ebf1923-1f21-455d-9520-2319f2b9f17c%2FEnzoRomero_2022U_VO_Intro.mp3


"(<Sentence: Prosthetics have advanced so much in recent years, but for amputees around the world, they're still often way too expensive.>, <Sentence: In his talk from TED 2022, bionic innovator Enzo Romero shares how he's found unconventional solutions to get more advanced prosthetics to the people who need them.>, <Sentence: Every week, I interview some of my favorite thinkers to learn how we can bring out the best in ourselves and others.>, <Sentence: Find Rethinking on Apple Podcasts, Spotify, Amazon Music, or wherever you listen.>, <Sentence: One in seven people around the world has some kind of disability.>, <Sentence: Being born without my right hand has defined who I am over the past 30 years.>, <Sentence: I was fortunate to grow in the sacred valley of the Incas in Cusco, Peru, surrounded by people who weren't surprised to see me driving my bike or probably playing basketball.>, <Sentence: But then being twice as stunned when the medical robots fitted in with a prosthesis that

## 4. Summarization using LexRank:

approach of LexRank is that a particular sentence is recommended by other similar sentences and hence is ranked higher.

Higher the rank, higher is the priority of being included in the summarized text.


In [20]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer


In [21]:
def get_lexrank_summary(link):
  transcript= get_transcripts(link)
  transcript= ''.join(transcript)
  parser=PlaintextParser.from_string(transcript,Tokenizer('english'))
    # Creating a summary of 3 sentences.
  lex_rank_summarizer = LexRankSummarizer()
  lexrank_summary = lex_rank_summarizer(parser.document,sentences_count=32)
  return str(lexrank_summary)
  # Printing the summary
  

In [22]:
lexrank_summary= get_lexrank_summary('https://podcasts.google.com/feed/aHR0cHM6Ly9mZWVkczIuZmVlZGJ1cm5lci5jb20vdGVkdGFsa3NfYXVkaW8/episode/ZW4uYXVkaW8udGFsay50ZWQuY29tOjk3NDA4?sa=X&ved=0CAUQkfYCahgKEwiQuOnlr7L6AhUAAAAAHQAAAAAQiwc')
print(lexrank_summary)

https://backend.endpoints.firstory-709db.cloud.goog/play.mp3?url=https%3A%2F%2Fdts.podtrac.com%2Fredirect.mp3%2Fdownload.ted.com%2Ftalks%2FEnzoRomero_2022U.mp3%3Fapikey%3D172BB350-0207%26prx_url%3Dhttps%3A%2F%2Fchtbl.com%2Ftrack%2F48D18%2Fhttps%3A%2F%2Fdovetail.prxu.org%2F70%2F1ebf1923-1f21-455d-9520-2319f2b9f17c%2FEnzoRomero_2022U_VO_Intro.mp3


"(<Sentence: Prosthetics have advanced so much in recent years, but for amputees around the world, they're still often way too expensive.>, <Sentence: He makes them more affordable and functional.>, <Sentence: One in seven people around the world has some kind of disability.>, <Sentence: When I was seven, I remember being stunned after I watched Star Wars, The Empire Strikes Back, when Luke's hand gets cut off, the same one I didn't have.>, <Sentence: But then being twice as stunned when the medical robots fitted in with a prosthesis that could move and feel.>, <Sentence: From then on, no greater purpose lived in my mind, which was to have a prosthesis like Luke Skywalker.>, <Sentence: This is one of the main reasons why it will cost at least $40,000 to make this prosthesis.>, <Sentence: But 40k is way too much, considering that four out of five people with upper limb amputation live in development countries.>, <Sentence: Even the $25,000 that costs a robotic prosthesis imported from a

## 5. Summarization using Luhn:

Luhn Summarization algorithm’s approach is based on TF-IDF (Term Frequency-Inverse Document Frequency). It is useful when very low frequent words as well as highly frequent words(stopwords) are both not significant.

In [23]:
# Import the summarizer
from sumy.summarizers.luhn import LuhnSummarizer
# Creating the parser
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser

In [24]:
def get_luhn_summary(link):
  transcript= get_transcripts(link)
  transcript= ''.join(transcript)
  parser=PlaintextParser.from_string(transcript,Tokenizer('english'))
      #  Creating the summarizer
  luhn_summarizer=LuhnSummarizer()
  luhn_summary=luhn_summarizer(parser.document,sentences_count=32)
  return str(luhn_summary)



In [25]:
luhn_summary= get_luhn_summary('https://podcasts.google.com/feed/aHR0cHM6Ly9mZWVkczIuZmVlZGJ1cm5lci5jb20vdGVkdGFsa3NfYXVkaW8/episode/ZW4uYXVkaW8udGFsay50ZWQuY29tOjk3NDA4?sa=X&ved=0CAUQkfYCahgKEwiQuOnlr7L6AhUAAAAAHQAAAAAQiwc')
print(luhn_summary)

https://backend.endpoints.firstory-709db.cloud.goog/play.mp3?url=https%3A%2F%2Fdts.podtrac.com%2Fredirect.mp3%2Fdownload.ted.com%2Ftalks%2FEnzoRomero_2022U.mp3%3Fapikey%3D172BB350-0207%26prx_url%3Dhttps%3A%2F%2Fchtbl.com%2Ftrack%2F48D18%2Fhttps%3A%2F%2Fdovetail.prxu.org%2F70%2F1ebf1923-1f21-455d-9520-2319f2b9f17c%2FEnzoRomero_2022U_VO_Intro.mp3
(<Sentence: Prosthetics have advanced so much in recent years, but for amputees around the world, they're still often way too expensive.>, <Sentence: In his talk from TED 2022, bionic innovator Enzo Romero shares how he's found unconventional solutions to get more advanced prosthetics to the people who need them.>, <Sentence: Find Rethinking on Apple Podcasts, Spotify, Amazon Music, or wherever you listen.>, <Sentence: One in seven people around the world has some kind of disability.>, <Sentence: I was fortunate to grow in the sacred valley of the Incas in Cusco, Peru, surrounded by people who weren't surprised to see me driving my bike or proba

## 6. Summarization using KL-Sum:

Another extractive method is the KL-Sum algorithm.

It selects sentences based on similarity of word distribution as the original text. It aims to lower the KL-divergence criteria (learn more). It uses greedy optimization approach and keeps adding sentences till the KL-divergence decreases.

In [26]:
from sumy.summarizers.kl import KLSummarizer
# Creating the parser
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser


In [37]:
def get_klsum_summary(link):
  transcript= get_transcripts(link)
  transcript= ''.join(transcript)
  parser=PlaintextParser.from_string(transcript,Tokenizer('english'))
  # Instantiating the  KLSummarizer
  kl_summarizer=KLSummarizer()
  kl_summary=kl_summarizer(parser.document,sentences_count=32)
  return str(kl_summary)
  

In [38]:
kl_summary= get_klsum_summary('https://podcasts.google.com/feed/aHR0cHM6Ly9mZWVkczIuZmVlZGJ1cm5lci5jb20vdGVkdGFsa3NfYXVkaW8/episode/ZW4uYXVkaW8udGFsay50ZWQuY29tOjk3NDA4?sa=X&ved=0CAUQkfYCahgKEwiQuOnlr7L6AhUAAAAAHQAAAAAQiwc')
print(kl_summary)

https://backend.endpoints.firstory-709db.cloud.goog/play.mp3?url=https%3A%2F%2Fdts.podtrac.com%2Fredirect.mp3%2Fdownload.ted.com%2Ftalks%2FEnzoRomero_2022U.mp3%3Fapikey%3D172BB350-0207%26prx_url%3Dhttps%3A%2F%2Fchtbl.com%2Ftrack%2F48D18%2Fhttps%3A%2F%2Fdovetail.prxu.org%2F70%2F1ebf1923-1f21-455d-9520-2319f2b9f17c%2FEnzoRomero_2022U_VO_Intro.mp3
(<Sentence: It's TED Talks Daily.>, <Sentence: Prosthetics have advanced so much in recent years, but for amputees around the world, they're still often way too expensive.>, <Sentence: Have you ever wondered what makes great minds tick?>, <Sentence: Find Rethinking on Apple Podcasts, Spotify, Amazon Music, or wherever you listen.>, <Sentence: One in seven people around the world has some kind of disability.>, <Sentence: From then on, no greater purpose lived in my mind, which was to have a prosthesis like Luke Skywalker.>, <Sentence: Since this was a research project, we had enough funding to buy stuff from abroad.>, <Sentence: This is one of th

# 7. Summarization using transformers
## Summarization using BERT
 This works by first embedding the sentences, then running a clustering algorithm, finding the sentences that are closest to the cluster's centroids

In [ ]:
 ! pip install bert-extractive-summarizer


In [30]:

from summarizer import Summarizer
model1= Summarizer()
model = whisper.load_model("medium")



Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [31]:
def get_bert_summary(link):
  transcript= get_transcripts(link)
  transcript= ''.join(transcript)
  return model1(transcript,ratio=0.5)

In [39]:
bert_summary= get_bert_summary('https://podcasts.google.com/feed/aHR0cHM6Ly9mZWVkczIuZmVlZGJ1cm5lci5jb20vdGVkdGFsa3NfYXVkaW8/episode/ZW4uYXVkaW8udGFsay50ZWQuY29tOjk3NDA4?sa=X&ved=0CAUQkfYCahgKEwiQuOnlr7L6AhUAAAAAHQAAAAAQiwc')
print(bert_summary)

https://backend.endpoints.firstory-709db.cloud.goog/play.mp3?url=https%3A%2F%2Fdts.podtrac.com%2Fredirect.mp3%2Fdownload.ted.com%2Ftalks%2FEnzoRomero_2022U.mp3%3Fapikey%3D172BB350-0207%26prx_url%3Dhttps%3A%2F%2Fchtbl.com%2Ftrack%2F48D18%2Fhttps%3A%2F%2Fdovetail.prxu.org%2F70%2F1ebf1923-1f21-455d-9520-2319f2b9f17c%2FEnzoRomero_2022U_VO_Intro.mp3


/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Prosthetics have advanced so much in recent years, but for amputees around the world, they're still often way too expensive. He makes them more affordable and functional. Have you ever wondered what makes great minds tick? I'm Adam Grant, and on my new podcast Rethinking, I'm trying to find the answers. Every week, I interview some of my favorite thinkers to learn how we can bring out the best in ourselves and others. Find Rethinking on Apple Podcasts, Spotify, Amazon Music, or wherever you listen. Since this was a research project, we had enough funding to buy stuff from abroad. So, for example, the motors here were all brought by Switzerland. It is often that people get injured while doing manual labor activities, like farming or metal manufacturing. Getting back to work and having access to money demands a functional hand. It took me time to realize what a huge privilege I had to study a career that would help fulfill my dreams. But actually, what was preventing me from developing a

## 8. Summarization using SBERT

In [40]:
! pip install -U sentence-transformers

NotImplementedError: ignored

In [41]:
from summarizer.sbert import SBertSummarizer
model1 = SBertSummarizer('paraphrase-MiniLM-L6-v2')
model = whisper.load_model("medium")



In [42]:
def get_sbert_summary(link):
  transcript= get_transcripts(link)
  transcript= ''.join(transcript)
  return model1(transcript, ratio=0.5)

In [43]:
sbert_summary= get_sbert_summary('https://podcasts.google.com/feed/aHR0cHM6Ly9mZWVkczIuZmVlZGJ1cm5lci5jb20vdGVkdGFsa3NfYXVkaW8/episode/ZW4uYXVkaW8udGFsay50ZWQuY29tOjk3NDA4?sa=X&ved=0CAUQkfYCahgKEwiQuOnlr7L6AhUAAAAAHQAAAAAQiwc')
print(sbert_summary)

https://backend.endpoints.firstory-709db.cloud.goog/play.mp3?url=https%3A%2F%2Fdts.podtrac.com%2Fredirect.mp3%2Fdownload.ted.com%2Ftalks%2FEnzoRomero_2022U.mp3%3Fapikey%3D172BB350-0207%26prx_url%3Dhttps%3A%2F%2Fchtbl.com%2Ftrack%2F48D18%2Fhttps%3A%2F%2Fdovetail.prxu.org%2F70%2F1ebf1923-1f21-455d-9520-2319f2b9f17c%2FEnzoRomero_2022U_VO_Intro.mp3


/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Prosthetics have advanced so much in recent years, but for amputees around the world, they're still often way too expensive. He makes them more affordable and functional. Have you ever wondered what makes great minds tick? I'm Adam Grant, and on my new podcast Rethinking, I'm trying to find the answers. One in seven people around the world has some kind of disability. I was fortunate to grow in the sacred valley of the Incas in Cusco, Peru, surrounded by people who weren't surprised to see me driving my bike or probably playing basketball. But then being twice as stunned when the medical robots fitted in with a prosthesis that could move and feel. I was able to move each finger individually, and also we developed a way in which we get to pass the sensation from the fingertips into my forearm using haptic feedback system. Since this was a research project, we had enough funding to buy stuff from abroad. So, for example, the motors here were all brought by Switzerland. Even the $25,000 t

# Analysing the generated summary using Rogue score & Bleu

## 1. Genism(Text rank algorithm)

In [ ]:
!pip install rouge
!pip install nltk


In [45]:
from rouge import Rouge
from nltk.translate.bleu_score import sentence_bleu


In [47]:
transcripts= get_transcripts('https://podcasts.google.com/feed/aHR0cHM6Ly9mZWVkczIuZmVlZGJ1cm5lci5jb20vdGVkdGFsa3NfYXVkaW8/episode/ZW4uYXVkaW8udGFsay50ZWQuY29tOjk3NDA4?sa=X&ved=0CAUQkfYCahgKEwiQuOnlr7L6AhUAAAAAHQAAAAAQiwc')
original_text= ''.join(transcripts)
text_tokens = nltk.word_tokenize(original_text)


https://backend.endpoints.firstory-709db.cloud.goog/play.mp3?url=https%3A%2F%2Fdts.podtrac.com%2Fredirect.mp3%2Fdownload.ted.com%2Ftalks%2FEnzoRomero_2022U.mp3%3Fapikey%3D172BB350-0207%26prx_url%3Dhttps%3A%2F%2Fchtbl.com%2Ftrack%2F48D18%2Fhttps%3A%2F%2Fdovetail.prxu.org%2F70%2F1ebf1923-1f21-455d-9520-2319f2b9f17c%2FEnzoRomero_2022U_VO_Intro.mp3


In [ ]:
len(original_text.split('.'))

In [48]:
textrank_summary= get_textrank_summary('https://podcasts.google.com/feed/aHR0cHM6Ly9mZWVkczIuZmVlZGJ1cm5lci5jb20vdGVkdGFsa3NfYXVkaW8/episode/ZW4uYXVkaW8udGFsay50ZWQuY29tOjk3NDA4?sa=X&ved=0CAUQkfYCahgKEwiQuOnlr7L6AhUAAAAAHQAAAAAQiwc')


https://backend.endpoints.firstory-709db.cloud.goog/play.mp3?url=https%3A%2F%2Fdts.podtrac.com%2Fredirect.mp3%2Fdownload.ted.com%2Ftalks%2FEnzoRomero_2022U.mp3%3Fapikey%3D172BB350-0207%26prx_url%3Dhttps%3A%2F%2Fchtbl.com%2Ftrack%2F48D18%2Fhttps%3A%2F%2Fdovetail.prxu.org%2F70%2F1ebf1923-1f21-455d-9520-2319f2b9f17c%2FEnzoRomero_2022U_VO_Intro.mp3


In [49]:
summary_tokens = nltk.word_tokenize(textrank_summary)

# Initialize the ROUGE evaluator
rouge = Rouge()

# Calculate the ROUGE scores
scores = rouge.get_scores(' '.join(summary_tokens), ' '.join(text_tokens))

# Print the ROUGE scores
print('ROUGE-1: {}'.format(scores[0]['rouge-1']['f']))
print('ROUGE-2: {}'.format(scores[0]['rouge-2']['f']))
print('ROUGE-L: {}'.format(scores[0]['rouge-l']['f']))

ROUGE-1: 0.7686746940627378
ROUGE-2: 0.714457826689476
ROUGE-L: 0.7686746940627378


## 3. LSA

In [50]:
lsa_summary= get_lsa_summary('https://podcasts.google.com/feed/aHR0cHM6Ly9mZWVkczIuZmVlZGJ1cm5lci5jb20vdGVkdGFsa3NfYXVkaW8/episode/ZW4uYXVkaW8udGFsay50ZWQuY29tOjk3NDA4?sa=X&ved=0CAUQkfYCahgKEwiQuOnlr7L6AhUAAAAAHQAAAAAQiwc')


https://backend.endpoints.firstory-709db.cloud.goog/play.mp3?url=https%3A%2F%2Fdts.podtrac.com%2Fredirect.mp3%2Fdownload.ted.com%2Ftalks%2FEnzoRomero_2022U.mp3%3Fapikey%3D172BB350-0207%26prx_url%3Dhttps%3A%2F%2Fchtbl.com%2Ftrack%2F48D18%2Fhttps%3A%2F%2Fdovetail.prxu.org%2F70%2F1ebf1923-1f21-455d-9520-2319f2b9f17c%2FEnzoRomero_2022U_VO_Intro.mp3


In [51]:
summary_tokens = nltk.word_tokenize(lsa_summary)

# Initialize the ROUGE evaluator
rouge = Rouge()

# Calculate the ROUGE scores
scores = rouge.get_scores(' '.join(summary_tokens), ' '.join(text_tokens))

# Print the ROUGE scores
print('ROUGE-1: {}'.format(scores[0]['rouge-1']['f']))
print('ROUGE-2: {}'.format(scores[0]['rouge-2']['f']))
print('ROUGE-L: {}'.format(scores[0]['rouge-l']['f']))

ROUGE-1: 0.7920560699543903
ROUGE-2: 0.6863905278502574
ROUGE-L: 0.7920560699543903


## 4.LexRank

In [52]:
lexrank_summary= get_lexrank_summary('https://podcasts.google.com/feed/aHR0cHM6Ly9mZWVkczIuZmVlZGJ1cm5lci5jb20vdGVkdGFsa3NfYXVkaW8/episode/ZW4uYXVkaW8udGFsay50ZWQuY29tOjk3NDA4?sa=X&ved=0CAUQkfYCahgKEwiQuOnlr7L6AhUAAAAAHQAAAAAQiwc')


https://backend.endpoints.firstory-709db.cloud.goog/play.mp3?url=https%3A%2F%2Fdts.podtrac.com%2Fredirect.mp3%2Fdownload.ted.com%2Ftalks%2FEnzoRomero_2022U.mp3%3Fapikey%3D172BB350-0207%26prx_url%3Dhttps%3A%2F%2Fchtbl.com%2Ftrack%2F48D18%2Fhttps%3A%2F%2Fdovetail.prxu.org%2F70%2F1ebf1923-1f21-455d-9520-2319f2b9f17c%2FEnzoRomero_2022U_VO_Intro.mp3


In [53]:
summary_tokens = nltk.word_tokenize(lexrank_summary)

# Initialize the ROUGE evaluator
rouge = Rouge()

# Calculate the ROUGE scores
scores = rouge.get_scores(' '.join(summary_tokens), ' '.join(text_tokens))

# Print the ROUGE scores
print('ROUGE-1: {}'.format(scores[0]['rouge-1']['f']))
print('ROUGE-2: {}'.format(scores[0]['rouge-2']['f']))
print('ROUGE-L: {}'.format(scores[0]['rouge-l']['f']))

ROUGE-1: 0.7058823483306574
ROUGE-2: 0.6150966891955468
ROUGE-L: 0.7058823483306574


## 5.Luhn

In [ ]:
luhn_summary= get_luhn_summary('https://podcasts.google.com/feed/aHR0cHM6Ly9mZWVkczIuZmVlZGJ1cm5lci5jb20vdGVkdGFsa3NfYXVkaW8/episode/ZW4uYXVkaW8udGFsay50ZWQuY29tOjk3NDA4?sa=X&ved=0CAUQkfYCahgKEwiQuOnlr7L6AhUAAAAAHQAAAAAQiwc')


In [54]:
summary_tokens = nltk.word_tokenize(luhn_summary)

# Initialize the ROUGE evaluator
rouge = Rouge()

# Calculate the ROUGE scores
scores = rouge.get_scores(' '.join(summary_tokens), ' '.join(text_tokens))

# Print the ROUGE scores
print('ROUGE-1: {}'.format(scores[0]['rouge-1']['f']))
print('ROUGE-2: {}'.format(scores[0]['rouge-2']['f']))
print('ROUGE-L: {}'.format(scores[0]['rouge-l']['f']))

ROUGE-1: 0.7878077325936871
ROUGE-2: 0.7249712265269528
ROUGE-L: 0.7878077325936871


## 6. KL_Sum

In [55]:
kl_summary= get_klsum_summary('https://podcasts.google.com/feed/aHR0cHM6Ly9mZWVkczIuZmVlZGJ1cm5lci5jb20vdGVkdGFsa3NfYXVkaW8/episode/ZW4uYXVkaW8udGFsay50ZWQuY29tOjk3NDA4?sa=X&ved=0CAUQkfYCahgKEwiQuOnlr7L6AhUAAAAAHQAAAAAQiwc')


https://backend.endpoints.firstory-709db.cloud.goog/play.mp3?url=https%3A%2F%2Fdts.podtrac.com%2Fredirect.mp3%2Fdownload.ted.com%2Ftalks%2FEnzoRomero_2022U.mp3%3Fapikey%3D172BB350-0207%26prx_url%3Dhttps%3A%2F%2Fchtbl.com%2Ftrack%2F48D18%2Fhttps%3A%2F%2Fdovetail.prxu.org%2F70%2F1ebf1923-1f21-455d-9520-2319f2b9f17c%2FEnzoRomero_2022U_VO_Intro.mp3


In [56]:
summary_tokens = nltk.word_tokenize(kl_summary)

# Initialize the ROUGE evaluator
rouge = Rouge()

# Calculate the ROUGE scores
scores = rouge.get_scores(' '.join(summary_tokens), ' '.join(text_tokens))

# Print the ROUGE scores
print('ROUGE-1: {}'.format(scores[0]['rouge-1']['f']))
print('ROUGE-2: {}'.format(scores[0]['rouge-2']['f']))
print('ROUGE-L: {}'.format(scores[0]['rouge-l']['f']))

ROUGE-1: 0.6268260248545614
ROUGE-2: 0.5127859985678083
ROUGE-L: 0.6268260248545614


## 7. BERT 

In [57]:
bert_summary= get_bert_summary('https://podcasts.google.com/feed/aHR0cHM6Ly9mZWVkczIuZmVlZGJ1cm5lci5jb20vdGVkdGFsa3NfYXVkaW8/episode/ZW4uYXVkaW8udGFsay50ZWQuY29tOjk3NDA4?sa=X&ved=0CAUQkfYCahgKEwiQuOnlr7L6AhUAAAAAHQAAAAAQiwc')


https://backend.endpoints.firstory-709db.cloud.goog/play.mp3?url=https%3A%2F%2Fdts.podtrac.com%2Fredirect.mp3%2Fdownload.ted.com%2Ftalks%2FEnzoRomero_2022U.mp3%3Fapikey%3D172BB350-0207%26prx_url%3Dhttps%3A%2F%2Fchtbl.com%2Ftrack%2F48D18%2Fhttps%3A%2F%2Fdovetail.prxu.org%2F70%2F1ebf1923-1f21-455d-9520-2319f2b9f17c%2FEnzoRomero_2022U_VO_Intro.mp3


/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
bert_summary

In [58]:
summary_tokens = nltk.word_tokenize(bert_summary)

# Initialize the ROUGE evaluator
rouge = Rouge()

# Calculate the ROUGE scores
scores = rouge.get_scores(' '.join(summary_tokens), ' '.join(text_tokens))

# Print the ROUGE scores
print('ROUGE-1: {}'.format(scores[0]['rouge-1']['f']))
print('ROUGE-2: {}'.format(scores[0]['rouge-2']['f']))
print('ROUGE-L: {}'.format(scores[0]['rouge-l']['f']))

ROUGE-1: 0.7063291093552636
ROUGE-2: 0.5935653273201068
ROUGE-L: 0.7063291093552636


## 8. SBERT

In [59]:
sbert_summary= get_sbert_summary('https://podcasts.google.com/feed/aHR0cHM6Ly9mZWVkczIuZmVlZGJ1cm5lci5jb20vdGVkdGFsa3NfYXVkaW8/episode/ZW4uYXVkaW8udGFsay50ZWQuY29tOjk3NDA4?sa=X&ved=0CAUQkfYCahgKEwiQuOnlr7L6AhUAAAAAHQAAAAAQiwc')


https://backend.endpoints.firstory-709db.cloud.goog/play.mp3?url=https%3A%2F%2Fdts.podtrac.com%2Fredirect.mp3%2Fdownload.ted.com%2Ftalks%2FEnzoRomero_2022U.mp3%3Fapikey%3D172BB350-0207%26prx_url%3Dhttps%3A%2F%2Fchtbl.com%2Ftrack%2F48D18%2Fhttps%3A%2F%2Fdovetail.prxu.org%2F70%2F1ebf1923-1f21-455d-9520-2319f2b9f17c%2FEnzoRomero_2022U_VO_Intro.mp3


/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [60]:
summary_tokens = nltk.word_tokenize(sbert_summary)

# Initialize the ROUGE evaluator
rouge = Rouge()

# Calculate the ROUGE scores
scores = rouge.get_scores(' '.join(summary_tokens), ' '.join(text_tokens))

# Print the ROUGE scores
print('ROUGE-1: {}'.format(scores[0]['rouge-1']['f']))
print('ROUGE-2: {}'.format(scores[0]['rouge-2']['f']))
print('ROUGE-L: {}'.format(scores[0]['rouge-l']['f']))

ROUGE-1: 0.7063291093552636
ROUGE-2: 0.5935653273201068
ROUGE-L: 0.7063291093552636
